<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
Prompt Variation
</div>

## Initial

In [66]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENROUTER_API_KEY = os.getenv("SHISHA_API_KEY")
GROQCLOUD_API_KEY = os.getenv("GROQCLOUD_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [67]:
import requests
from openai import OpenAI 

OPENROUTER_CLIENT = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

OPENAI_CLIENT = OpenAI(api_key=OPENAI_API_KEY)

def call_openai_api(prompt: str) -> str:
    try:
        response = OPENAI_CLIENT.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        )

        reply = response.choices[0].message.content

        return reply 
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None

def call_openrouter_api(prompt: str) -> str:
    try:
        completion = OPENROUTER_CLIENT.chat.completions.create(
            model="moonshotai/kimi-dev-72b:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        return completion.choices[0].message.content
    
    except Exception as e:
        print(f"API called failed: {e}")
        return None
    
def call_groq_api(prompt: str) -> str:
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQCLOUD_API_KEY}"
    }
    payload = {
        "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        return result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"GROQ API call failed: {e}")
        return None

In [68]:
AUGMENTATION_PROMPT = """Paraphase with the same meaning as this question:
- Keep the same language as the input question.
- 3 command-style questions with an incomplete sentence structure (no need for full subject-verb)
- 2 short, concise questions, but the hotel name must be included.
Output: a list of 5 string (no explanation, no extra text, remove ordinal number).
The question: {question}
"""

## Top-3 question-answer

In [69]:
import json

file_path = "./0-preprocessing.json"
list_questions = []
list_answers = []
list_hotels_name = []

with open(file_path, 'r', encoding='utf-8') as file:
    raw_data = json.load(file)

for data in raw_data:
    list_questions.append(data['conversations'][0]['value'])
    list_answers.append(data['conversations'][1]['value'])
    list_hotels_name.append(data['hotel_name'])

for question, answer, hotel_name in zip(list_questions[:3], list_answers[:3], list_hotels_name[:3]):
    print(hotel_name)
    print(question)
    print(answer)
    print("\n===\n")

Renaissance Riverside Saigon Hotel
Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?
Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.

===

Renaissance Riverside Saigon Hotel
Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?
Renaissance Riverside Saigon Hotel tọa lạc tại địa chỉ 8 - 15, Ton Duc Thang, Quận 1, Thành phố Hồ Chí Minh, Việt Nam.

===

Renaissance Riverside Saigon Hotel
Bạn có thể cho tôi biết các hạng phòng hiện có tại Renaissance Riverside Saigon Hotel không?
Khách sạn Renaissance Riverside Saigon có nhiều hạng phòng đa dạng như: Standard Room, Suite, Guest Room, Club Guest Room, Executive Studio, Executive Suite, R Club Room, D

## Send each question to LLM for variation

In [70]:
from typing import Dict, List

def add_variation_prompts(list_questions: List[str], answer: str, hotel_name: str) -> List[Dict]:
    new_conversations = []

    for question in list_questions:
        temp_conversation = {
            'conversations': [
                {
                    'from': 'human',
                    'value': question
                },
                {
                    'from': 'gpt',
                    'value': answer
                }
            ],
            'hotel_name': hotel_name
        }

        new_conversations.append(temp_conversation)

    return new_conversations

def extract_list_questions(response: str) -> List[str]:
    list_questions = response.split('\n')

    list_questions = [q.lstrip('-').strip() for q in list_questions if q.strip()]

    return list_questions

In [ ]:
list_conversations = []

for question, answer, hotel_name in zip(list_questions, list_answers, list_hotels_name):
    prompt = AUGMENTATION_PROMPT.format(question=question)
    response = call_openai_api(prompt)
    try:
        list_questions = extract_list_questions(response)
    except Exception as e:
        print(f"Error extracting questions: {e}")
        continue
    new_conversations = add_variation_prompts(list_questions, answer, hotel_name)
    
    print(question)
    print(f"{response}\n===\n")  # Debugging output to check the response

    list_conversations.extend(new_conversations)
    print(f"🦉 Current length of list_conversations: {len(list_conversations)}")

len(list_conversations)

Bạn có thể cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel được không?
- Cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel
- Cho tôi biết về Renaissance Riverside Saigon Hotel
- Thông tin chung về Renaissance Riverside Saigon Hotel
- Renaissance Riverside Saigon Hotel là gì?
- Renaissance Riverside Saigon Hotel có gì nổi bật?
===

Khách sạn Renaissance Riverside Saigon nằm ở địa chỉ nào vậy?
- Tìm địa chỉ của khách sạn Renaissance Riverside Saigon.  
- Địa chỉ khách sạn Renaissance Riverside Saigon ở đâu?  
- Vị trí của khách sạn Renaissance Riverside Saigon tại đâu?  
- Khách sạn Renaissance Riverside Saigon có ở đâu?  
- Renaissance Riverside Saigon khách sạn nằm ở đâu?
===

Bạn có thể cho tôi biết các hạng phòng hiện có tại Renaissance Riverside Saigon Hotel không?
- Cho tôi biết các hạng phòng tại Renaissance Riverside Saigon Hotel  
- Cung cấp danh sách các loại phòng ở Renaissance Riverside Saigon Hotel  
- Thông báo các hạng phòng hiện có tại 

1493

In [72]:
list_conversations[:2]  # Display the first two conversations to verify

[{'conversations': [{'from': 'human',
    'value': 'Cung cấp thông tin tổng quan về Renaissance Riverside Saigon Hotel'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham quan nổi tiếng như Nhà hát Lớn, Chợ Bến Thành.'}],
  'hotel_name': 'Renaissance Riverside Saigon Hotel'},
 {'conversations': [{'from': 'human',
    'value': 'Cho tôi biết về Renaissance Riverside Saigon Hotel'},
   {'from': 'gpt',
    'value': 'Renaissance Riverside Saigon Hotel là khách sạn 5 sao tọa lạc ngay trung tâm Thành phố Hồ Chí Minh. Khách sạn có 336 phòng với cửa sổ nhìn ra thành phố, nhiều tiện nghi hiện đại cùng 2 nhà hàng và bar sang trọng. Vị trí thuận tiện cách sân bay Tân Sơn Nhất khoảng 30 phút lái xe và gần các điểm tham 

## Save to file

In [73]:
def save_to_file(data: List[Dict], filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("[")

        for index, entry in enumerate(data):
            char = ','
            if index == len(data) - 1:
                char = ''

            file.write(json.dumps(entry, ensure_ascii=False) + f"{char}\n")

        file.write("]")
        
    print(f'Saved {len(data)} entries to {filename}')

In [74]:
save_to_file(list_conversations, '1-prompt_variation.json')

Saved 1493 entries to 1-prompt_variation.json
